In [126]:
import os
import re
import csv
import nltk
import numpy
import codecs
import nltk.data
import numpy as np
import pandas as pd
from math import log
from Bio import Medline
from scipy import spatial
from random import shuffle
#from pattern.en import tag
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn import feature_extraction
from nltk.stem import WordNetLemmatizer
from contractions import CONTRACTION_MAP
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from gensim.models.keyedvectors import KeyedVectors
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline

In [2]:
def preprocess(string):
    # Preprocesses a string by lowercasing, trimming, and removing non-alphanumeric
    return " ".join(re.findall(r'\w+', string, flags=re.UNICODE)).lower()

In [3]:
def normalize(word):
    return WNL.lemmatize(word).lower()

In [4]:
def remove_stopwords(lst):
    # Removing stopwords from a list of tokens
    return [w for w in lst if w not in feature_extraction.text.ENGLISH_STOP_WORDS]

In [5]:
def tokenized_lemmas(string):
    # Returns tokenized words
    return [normalize(token) for token in remove_stopwords(nltk.word_tokenize(string))]

In [6]:
WNL = WordNetLemmatizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get', 'tell', 'listen', 'one', 'two', 'three', 
'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero', 'join', 'find', 'make', 'say', 'ask','tell', 'see', 'try', 'back', 'also']
stopword_pubmed = stopword_list + ['a', 'about', 'again', 'all', 'almost', 'also', 'although', 'always', 'among', 'an', 'and', 
'another', 'any', 'are', 'as', 'at','be', 'because', 'been', 'before', 'being', 'between', 'both', 'but', 'by','can', 'could',
'did', 'do', 'does', 'done', 'due', 'during','each', 'either', 'enough', 'especially', 'etc','for', 'found', 'from', 'further',
'had', 'has', 'have', 'having', 'here', 'how', 'however','i', 'if', 'in', 'into', 'is', 'it', 'its', 'itself','just','kg', 'km',
'made', 'mainly', 'make', 'may', 'mg', 'might', 'ml', 'mm', 'most', 'mostly', 'must','nearly', 'neither', 'no', 'nor','obtained', 
'of', 'often', 'on', 'our', 'overall','perhaps', 'pmid','quite','rather', 'really', 'regarding','seem', 'seen', 'several', 'should', 
'show', 'showed', 'shown', 'shows', 'significantly', 'since', 'so', 'some', 'such', 'than', 'that', 'the', 'their', 'theirs', 
'them', 'then', 'there', 'therefore', 'these', 'they', 'this', 'those', 'through', 'thus', 'to','upon', 'use', 'used', 'using',
'various', 'very', 'was', 'we', 'were', 'what', 'when', 'which', 'while', 'with', 'within', 'without', 'would']

In [7]:
df = pd.DataFrame()
filelist = os.listdir(r"Task 2\training\extracted_data")
for datei in filelist:
    if datei.endswith('.pids'):
        frame = pd.read_table(r"Task 2\training\extracted_data\\"+datei, names = ['PMID'], encoding='utf8')
        df = df.append(frame)
df = pd.DataFrame(df.PMID.str.split(' ', 1).tolist(), columns=['TOPIC','PMID'])
#LISTE mit PMIDS erstellen
pmids = df['PMID'].tolist()
# GET Abstracts 
pmid_list = []
title_list = []
ab_list = []

In [24]:
#Save Pmids, Titles and Abstracts to the Lists
with open("pubmed.txt") as handle:
    record = Medline.read(handle)
    for record in Medline.parse(handle):
        ab_list.append(record.get("AB"))
        title_list.append(record.get("TI"))
        pmid_list.append(record.get("PMID"))
#Build Dicts from the Information       
pmid_title = dict(zip(pmid_list,title_list))
pmid_ab = dict(zip(pmid_list,ab_list))
#REMOVE Entrys with None Values
pmid_ab_final = dict((k, v) for k, v in pmid_ab.items() if v)
pmid_list2=[]
ab_list2=[]
pmid_list2=list(pmid_ab_final.keys())
ab_list2=list(pmid_ab_final.values())
df2 = pd.DataFrame(list(zip(pmid_list,ab_list)))
df2.columns=['PMID','ABSTRACT']

In [25]:
#CREATE TOPIC und SYSTEMATIC REVIEW TITLE from the .TITLE Files
df3 = pd.DataFrame()
filelist = os.listdir(r"Task 2\training\extracted_data")
for datei in filelist:
    if datei.endswith('.title'):
        frame = pd.read_table(r"Task 2\training\extracted_data\\"+datei, names = ['TITLE'], encoding='utf8')
        df3 = df3.append(frame)
df3 = pd.DataFrame(df3.TITLE.str.split(' ',1).tolist(), columns = ['TOPIC','TITLE'])
df3.columns= ['TOPIC','SYS TITLE']
topics = df['TOPIC'].tolist()
titles = df3['SYS TITLE'].tolist()
print("initialized...")

initialized...


In [29]:
def text2count(body):
    freq = defaultdict(int)
    for word in body:
        freq[word] += 1
    return freq

In [30]:
def sentences2vectors(sentences):
    body_vectors = []
    for sentence in sentences:
        words = sentence
        cnt = 0
        body_vector = np.array([0.0] * 300)
        for word in words:
            if word in word_vectors:
                body_vector = body_vector + word_vectors[word]
                cnt += 1
        if cnt > 0:
            body_vector /= cnt
            body_vector /= np.linalg.norm(body_vector)
        body_vectors.append(body_vector)
    return body_vectors

In [88]:
def cosine_sim(title, body):
    title_vectors = sentences2vectors([title])
    title_vector = title_vectors[0]
    body_vectors = sentences2vectors(body)
    
    max_sim = -1
    diff_vector = title_vector
    for body_vector in body_vectors:
        similarity = 1 - spatial.distance.cosine(title_vector, body_vector)
        if similarity > max_sim:
            max_sim = similarity
            diff_vector = title_vector - body_vector
    features = [max_sim]
    for v in diff_vector:
        features.append(v)
    return max_sim

In [89]:
def lexical_overlaps(title, body, idf):
    features = []

    words_in_body = text2count(body)
    words_in_title = text2count(title)

    maximum, maximum_cnt = 0.0, 0.0
    for (word, cnt_title) in words_in_title.items():
        maximum += cnt_title * idf[word]
        maximum_cnt += cnt_title

    overlaps, overlap_cnt = 0, 0
    for (word, cnt_title) in words_in_title.items():
        if word in words_in_body:
            tf = min(cnt_title, words_in_body[word])
            overlap_cnt += tf
            overlaps += tf * idf[word]
    features += [overlaps, overlaps / (maximum+0.001), overlap_cnt, overlap_cnt / (maximum_cnt+0.001)]

    words_in_body = text2count(body[:len(title) * 4])
    overlaps, overlap_cnt = 0, 0
    for (word, cnt_title) in words_in_title.items():
        if word in words_in_body:
            tf = min(cnt_title, words_in_body[word])
            overlap_cnt += tf
            overlaps += tf * idf[word]
    features += [overlaps, overlaps / (maximum+0.001), overlap_cnt, overlap_cnt / (maximum_cnt+0.001)]

    return overlaps

In [33]:
def title2vector(title, word2vec, idf):
    vector = np.array([0.0] * 300)
    cnt = 0
    for word in title:
        if word in word2vec:
            vector += word2vec[word]
            cnt += 1
    if cnt > 0:
        vector /= cnt
        vector /= np.linalg.norm(vector)
    return vector

In [34]:
def compute_overlap(title, body_sentence, idf):
    words_in_body = text2count(body_sentence)
    words_in_title = text2count(title)

    maximum, maximum_cnt = 0.0, 0.0
    for (word, cnt_title) in words_in_title.items():
        maximum += cnt_title * idf[word]
        maximum_cnt += cnt_title

    overlaps, overlap_cnt = 0, 0
    for (word, cnt_title) in words_in_title.items():
        if word in words_in_body:
            tf = min(cnt_title, words_in_body[word])
            overlap_cnt += tf
            overlaps += tf * idf[word]
    return overlaps / (maximum_cnt+0.001), overlap_cnt / (maximum_cnt+0.001)

In [92]:
def semantic_similarities(title, body_sentences, word2vec, idf):
    max_overlap, max_overlap_cnt = 0, 0
    #title_vector = title2vector(title, word2vec, idf)
    max_sim = -1
    best_vector = np.array([0.0] * 300)

    supports = []
    for sub_body in body_sentences:
        #sub_body_vector = title2vector(sub_body, word2vec, idf)

        cur_overlap, cur_overlap_cnt = compute_overlap(title, sub_body, idf)

        max_overlap = max(max_overlap, cur_overlap)
        max_overlap_cnt = max(max_overlap_cnt, cur_overlap_cnt)

        similarity = 0
#         for i in range(300):
#             similarity += title_vector[i] * sub_body_vector[i]
#         if similarity > max_sim:
#             max_sim = similarity
            #best_vector = sub_body_vector

        #supports.append(similarity)

    #features = [max_overlap, max_overlap_cnt, max(supports), min(supports)]

    #for v in title_vector - best_vector:
    #    features.append(v)
#     for v in best_vector:
#         features.append(v)
#     for v in title_vector:
#         features.append(v)
    return max_overlap

In [36]:
word2vec = {}
print ('loading word2vec...')
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
for word in word_vectors.vocab:
    word2vec[normalize(word)] = word_vectors[word]
print ('word2vec loaded')

loading word2vec...
word2vec loaded


In [37]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [85]:
def extract_features(title, abstract, idf, word2vec):
    return [lexical_overlaps(title, abstract, idf), semantic_similarities(title, abstract, word2vec, idf), cosine_sim(title, abstract), levenshtein(title, abstract), lexical_overlaps(title, abstract, idf) + semantic_similarities(title, abstract, word2vec, idf) + cosine_sim(title, abstract) + levenshtein(title, abstract)]

In [39]:
qrel_file = pd.read_table(r"Task 2/training/qrels/qrel_abs_train", header=None, sep='\s*')

C:\Python36AMD64\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\Python36AMD64\lib\site-packages\pandas\io\parsers.py:2227: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
C:\Python36AMD64\lib\site-packages\pandas\io\parsers.py:2229: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [40]:
final_list = pd.DataFrame()

In [41]:
qrel_lists = np.array(qrel_file)

In [42]:
qrel_lists[0]

array(['CD010438', 0, 4461416, 0], dtype=object)

In [43]:
titles_list = np.array(df3)

In [44]:
titles_dict = {}
for i in range(len(titles_list)):
    titles_dict[titles_list[i][0]] = titles_list[i][1]

In [45]:
print(titles_dict['CD011984'])

Urine tests for Down's syndrome screening


In [46]:
qrel_file = np.array(qrel_file, dtype=str)
qrel_file

array([['CD010438', '0', '4461416', '0'],
       ['CD010438', '0', '19762299', '0'],
       ['CD010438', '0', '16607076', '0'],
       ...,
       ['CD007394', '0', '2194959', '0'],
       ['CD007394', '0', '1633294', '0'],
       ['CD007394', '0', '2532389', '0']], dtype='<U64')

In [47]:
pmid_list = np.array(df2)

In [48]:
pmid_dict = {}
for i in range(len(pmid_list)):
    pmid_dict[pmid_list[i][0]] = pmid_list[i][1]

In [49]:
topic_titles = []
pmid_abstracts = []
relevancy_labels = []
for i in range(len(qrel_file)):
    if qrel_file[i][0] in titles_dict and qrel_file[i][2] in pmid_dict and pmid_dict[qrel_file[i][2]] != 'None':
        topic_titles.append(titles_dict[qrel_file[i][0]])
        pmid_abstracts.append(pmid_dict[qrel_file[i][2]])
        relevancy_labels.append(qrel_file[i][3])

In [50]:
dataset_frame = pd.DataFrame({'Topics': topic_titles, 'Abstracts': pmid_abstracts, 'Labels': relevancy_labels})

In [51]:
dataset_frame.dropna(how='any', axis=0, inplace=True)

In [53]:
#Now all the rows with None values have been removed.
pmid_abstracts = np.array(dataset_frame.Abstracts)
topic_titles = np.array(dataset_frame.Topics)
relevancy_labels = np.array(dataset_frame.Labels)

In [54]:
print(len(pmid_abstracts), len(topic_titles), len(relevancy_labels))

118593 118593 118593


In [55]:
idf = defaultdict(float)
for content in pmid_abstracts:
    #print(content)
    for word in set(tokenized_lemmas(content)):
        idf[word] += 1

In [56]:
for word in idf:
    idf[word] = log(len(pmid_abstracts) / idf[word])

In [57]:
train_X = []

In [58]:
print(len(pmid_abstracts), len(topic_titles), len(relevancy_labels))

118593 118593 118593


In [59]:
# for i in range(1000):
#     train_X.append(extract_features(abstract=pmid_abstracts[i], idf=idf, title=topic_titles[i], word2vec=word2vec))
# Uncomment the code if the features are being calculated for the first time, otherwise, we'll read them from the pickle file.

In [60]:
import pickle
train_X = pickle.load(open("trainX.pkl", "rb"))

In [61]:
print(len(train_X))

118693


In [62]:
#train_X = train_X[100:]

In [63]:
print(len(train_X))

118593


In [65]:
df_test = pd.DataFrame()
filelist_test = os.listdir(r"Task 2\testing\extracted_data")
for datei in filelist_test:
    if datei.endswith('.pids'):
        frame = pd.read_table(r"Task 2\testing\extracted_data\\"+datei, names = ['PMID'], encoding='utf8')
        df_test = df_test.append(frame)
df_test = pd.DataFrame(df_test.PMID.str.split(' ', 1).tolist(), columns=['TOPIC','PMID'])
#LISTE mit PMIDS erstellen
pmids_test = df_test['PMID'].tolist()
# GET Abstracts 
pmid_list_test = []
title_list_test = []
ab_list_test = []

In [66]:
#Save Pmids, Titles and Abstracts to the Lists
with open("pubmed.txt") as handle:
    record = Medline.read(handle)
    for record in Medline.parse(handle):
        ab_list_test.append(record.get("AB"))
        title_list_test.append(record.get("TI"))
        pmid_list_test.append(record.get("PMID"))
#Build Dicts from the Information       
pmid_title_test = dict(zip(pmid_list_test,title_list_test))
pmid_ab_test = dict(zip(pmid_list_test,ab_list_test))
#REMOVE Entrys with None Values
pmid_ab_final_test = dict((k, v) for k, v in pmid_ab_test.items() if v)
pmid_list2_test=[]
ab_list2_test=[]
pmid_list2_test=list(pmid_ab_final_test.keys())
ab_list2_test=list(pmid_ab_final_test.values())
df2_test = pd.DataFrame(list(zip(pmid_list_test,ab_list_test)))
df2_test.columns=['PMID','ABSTRACT']

In [67]:
#CREATE TOPIC und SYSTEMATIC REVIEW TITLE from the .TITLE Files
df3_test = pd.DataFrame()
filelist_test = os.listdir(r"Task 2\testing\extracted_data")
for datei in filelist_test:
    if datei.endswith('.title'):
        frame = pd.read_table(r"Task 2\testing\extracted_data\\"+datei, names = ['TITLE'], encoding='utf8')
        df3_test = df3_test.append(frame)
df3_test = pd.DataFrame(df3_test.TITLE.str.split(' ',1).tolist(), columns = ['TOPIC','TITLE'])
df3_test.columns= ['TOPIC','SYS TITLE']
topics_test = df_test['TOPIC'].tolist()
titles_test = df3_test['SYS TITLE'].tolist()
print("initialized...")

initialized...


In [68]:
qrel_file_test = pd.read_table(r"Task 2/testing/qrels/qrel_abs_test.txt", header=None, sep='\s*')

C:\Python36AMD64\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\Python36AMD64\lib\site-packages\pandas\io\parsers.py:2227: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
C:\Python36AMD64\lib\site-packages\pandas\io\parsers.py:2229: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [69]:
final_list_test = pd.DataFrame()

In [70]:
qrel_lists_test = np.array(qrel_file_test)

In [71]:
qrel_lists_test[0]

array(['CD007431', 0, 7072537, 0], dtype=object)

In [72]:
titles_list_test = np.array(df3_test)

In [73]:
titles_dict_test = {}
for i in range(len(titles_list_test)):
    titles_dict_test[titles_list_test[i][0]] = titles_list_test[i][1]

In [75]:
qrel_file_test = np.array(qrel_file_test, dtype=str)
qrel_file_test

array([['CD007431', '0', '7072537', '0'],
       ['CD007431', '0', '8748845', '0'],
       ['CD007431', '0', '3819738', '0'],
       ...,
       ['CD009925', '0', '15298970', '0'],
       ['CD009925', '0', '10377014', '0'],
       ['CD009925', '0', '2664755', '0']], dtype='<U64')

In [76]:
pmid_list_test = np.array(df2_test)

In [77]:
pmid_dict_test = {}
for i in range(len(pmid_list_test)):
    pmid_dict_test[pmid_list_test[i][0]] = pmid_list_test[i][1]

In [78]:
topic_titles_test = []
pmid_abstracts_test = []
relevancy_labels_test = []
for i in range(len(qrel_file_test)):
    if qrel_file_test[i][0] in titles_dict_test and qrel_file_test[i][2] in pmid_dict_test and pmid_dict_test[qrel_file_test[i][2]] != 'None':
        topic_titles_test.append(titles_dict_test[qrel_file_test[i][0]])
        pmid_abstracts_test.append(pmid_dict_test[qrel_file_test[i][2]])
        relevancy_labels_test.append(qrel_file_test[i][3])

In [79]:
dataset_frame_test = pd.DataFrame({'Topics': topic_titles_test, 'Abstracts': pmid_abstracts_test, 'Labels': relevancy_labels_test})

In [80]:
dataset_frame_test.dropna(how='any', axis=0, inplace=True)

In [81]:
#Now all the rows with None values have been removed.
pmid_abstracts_test = np.array(dataset_frame_test.Abstracts)
topic_titles_test = np.array(dataset_frame_test.Topics)
relevancy_labels_test = np.array(dataset_frame_test.Labels)

In [82]:
print(len(pmid_abstracts_test), len(topic_titles_test), len(relevancy_labels_test))

21872 21872 21872


In [83]:
test_X = []

In [93]:
# for i in range(len(pmid_abstracts_test)):
#     test_X.append(extract_features(abstract=pmid_abstracts_test[i], idf=idf, title=topic_titles_test[i], word2vec=word2vec))
#Dont need these if you have the testX.pkl file.

C:\Python36AMD64\lib\site-packages\scipy\spatial\distance.py:649: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [138]:
test_X = pickle.load(open("testX.pkl", "rb"))

In [139]:
clf = XGBClassifier(learning_rate=0.01, silent=False, base_score=0.1)

In [140]:
relevancy_labels_int = np.array(relevancy_labels, dtype=int)

In [141]:
relevancy_labels_test_int = np.array(relevancy_labels_test, dtype=int)

In [142]:
clf.fit(np.array(train_X), relevancy_labels_int)

XGBClassifier(base_score=0.1, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [143]:
clf.score(test_X, relevancy_labels_test_int)

C:\Python36AMD64\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9685899780541332

In [144]:
preds = clf.predict(test_X)

C:\Python36AMD64\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [145]:
np.sum(preds)

0

In [146]:
np.sum(relevancy_labels_int)

2319

In [147]:
np.sum(relevancy_labels_test_int)

687

In [157]:
clf2 = MLPClassifier(activation='logistic', hidden_layer_sizes=(2), alpha=0.000001)

In [158]:
clf2.fit(train_X, relevancy_labels_int)

MLPClassifier(activation='logistic', alpha=1e-06, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=2, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [159]:
preds2 = clf2.predict(test_X)

In [160]:
np.sum(preds2)

0

In [152]:
pklFil = open("testX.pkl", "wb")
pickle.dump(obj=test_X, file=pklFil)
pklFil.close()